In [1]:
# El Gamal Digital Signature
p = 37
Fp = FiniteField(p)

m = 14
g = 2
r = 19
sig = 19
pk = 23

print('got:', Fp(pk)**r * Fp(r) ** sig)
print('expected:', Fp(g) ** m)


got: 30
expected: 30


In [2]:
# check if g is a generator of Fp*
def is_generator(p: int, g: int, verbose: bool = False) -> bool:
    genereted_numbers = set()
    for i in range(p):
        genereted_numbers.add((g ** i) % p)

    not_generated = set(range(1, p - 1)) - genereted_numbers
    if verbose:
        print(not_generated)
        print(len(not_generated))
        
    return len(not_generated) == 0


# p = 31
# g = 2
# is_generator(p, g)


# find all multiplicative generators
generators = []
for g in range(1, 31):
    if is_generator(p, g):
        generators.append(g)

print(generators)


[2, 5, 13, 15, 17, 18, 19, 20, 22, 24]


In [3]:
# another way to find one generator
FiniteField(p).multiplicative_generator()

2

In [4]:
# online solver: https://asecuritysite.com/principles/log_poh?val1=7531&val2=6&val3=8101
def pohlig_helman(p: int, g: int, g_a: int) -> int:
    """
    find a satisfyinh g^a = g_a mod p
    """
    Fp = FiniteField(p)
    g_a = Fp(g_a)
    g = Fp(g)    
    
    mod_results = [] # a mod4, a mod9
    
    prime_to_power = list(factor(p - 1))  # list of tuples [ (prime_number, power), ... ]
    if len(prime_to_power) != 2:
        raise NotImplementedError(f'Dont know how to solve with {len(prime_to_power)} factors')
    
    for prime, power in prime_to_power:
        # a mod prime^power in base prime. for instance, 31 mod 27 in base 3 is: 
        # a0*3^0 + a1*3^1 + a2*e^2  = 1*3^0 + 1*3^1 + 0*e^2  so the array contains [1, 1, 0]
        
        
        
        basis_coeffs = [] # b_i values
        for i in range(1, power + 1):  # TODO: not sure about the range
            lhs = g_a ^ ((p-1) / (prime^i)) 
            
            # get RHS coeff
            rhs_coeff_exp = 0
            for j, coeff in enumerate(basis_coeffs):
                rhs_coeff_exp += coeff * (p-1) / (prime ^ (i - j))
            rhs_coeff = g ^ rhs_coeff_exp
            
            # iterate over possible RHS
            for possible_b_l in range(prime):
                rhs = rhs_coeff * (g ^ (possible_b_l * (p-1) / (prime ^ (i - len(basis_coeffs)))))
                if rhs == lhs:
                    basis_coeffs.append(possible_b_l)
                    break
                    
                if possible_b_l == prime - 1:
                    raise Exception('oops. probably bug. didnt break')
        
        mod_res = 0
        for i, basis_coeff in enumerate(basis_coeffs):
            mod_res += basis_coeff * (prime ^ i)
            
        mod_results.append(mod_res)
        
        
    # print(mod_results)

    n, m = prime_to_power[0][0] ^ prime_to_power[0][1], prime_to_power[1][0] ^ prime_to_power[1][1]
    one, c, d = xgcd(n, m)
    assert c * n + d * m == one == 1
    
    a = (mod_results[0] * d * m + mod_results[1] * c * n) % (p - 1)
    print(mod_results)
    print(f'({mod_results[0]} * {d} * {m} + {mod_results[1]} * {c} * {n}) % (p - 1)')
    return a



# tests:
p = 37
g = 2
g_a = 17
assert pohlig_helman(p, g, g_a) == 7

p = 23
g = 5
g_a = 8
a = pohlig_helman(p, g, g_a)
assert FiniteField(p)(g) ^ a == g_a


p = 12289
g = 11
g_a = 8080
a = pohlig_helman(p, g, g_a)
assert FiniteField(p)(g) ^ a == g_a
print(a)


# # 3 factors:
# p = 43
# g = 5
# g_a = 8
# assert pohlig_helman(p, g, g_a) == 15



[3, 7]
[0, 6]
[3342, 0]
3342


In [5]:
def lab06_6(n: int):
    p = n.next_prime()
    Fp = FiniteField(p)
    g = Fp.multiplicative_generator()
    h = Fp.random_element(x=1)
    
    print('p: ', p)
    print('g: ', g)
    print('h: ', h)
    print('a: ', pohlig_helman(p, g, h))
    
    
lab06_6(23112)

p:  23117
g:  2
h:  432
[3, 4555]
a:  4555


In [6]:
def lab06_6b(n: int):
    p = n.next_prime()
    Fp = FiniteField(p)
    return Fp.multiplicative_generator()


In [7]:
Fp = FiniteField(p)
Fp.multiplicative_generator()


11